# GDC April 2021 Webinar: Using the GDC API

### Monday, April 26, 2021<br>2:00 PM - 3:00 PM (EST)<br>Bill Wysocki, Director of User Services <br>University of Chicago

## API User's Guide and Other GDC Helpful Links

[GDC API User's Guide](https://docs.gdc.cancer.gov/API/Users_Guide/Getting_Started/)

[GDC Support Website](https://gdc.cancer.gov/support)

GDC Helpdesk E-mail - support@nci-gdc.datacommons.io

Others?

# Notebook Overview 

- [GDC API Overview](#api_overview')
- [GDC API Key Features](#api_features')
- [Jupyter Notebook Overvie](#jn_overview')
- [How to Search and Retrieve Data with GDC API](#search_retrieve)
    - [Retrieve() function]()
    - [Example 1: ]()
- [How to Download Files with GDC API](#download)
    - [Download]()
- [How to Analyze Data with GDC API](#analyze)
- [How to Perform BAM Slicing with GDC API](#bam_slice)
- [How to Submit Data with GDC API](#submit)


# <a id='api_overview'>GDC API Overview</a>

# <a id='api_features'>GDC API Key Features</a>

RESTful API - link to definition?

# <a id='jn_overview'>Jupyter Notebook Overview</a>

- links to JN resources? 
- Quick, couple sentence/1 paragraph summary
- Dependencies
- How to install dependencies from command line (or include a requirements.txt file?)
    - from windows, linux and macOS?


In [ ]:
#import packages
import requests
import json
import urllib

# <a id='search_retrieve'>How to Search and Retrieve Data with GDC API</a>


# <a id='download'>How to Download Files with GDC API</a>

# <a id='analyze'>How to Analyze Data with GDC API</a>

# <a id='bam_slice'>How to Perform BAM Slicing with GDC API</a>

# <a id='submit'>How to Submit Data with GDC API</a>